In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from IPython import display as ds
import matplotlib.image as mpimg
import cv2
import tensorflow as tf
import math

%matplotlib inline

/home/kvpcloud/.conda/envs/p3_gpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class Dataset:
    main = 'dataset/UCSD_Anomaly_Dataset.v1p2'
    latent_train = np.load('{}/latent_train.npy'.format(main))
    #test_images = np.load('{}/test.npy'.format(main))

In [3]:
print(Dataset.latent_train)

[[0.09196392 0.0478623  0.13000081 ... 0.10197813 0.         0.03637807]
 [0.11980576 0.07599361 0.08409608 ... 0.11578386 0.         0.01681237]
 [0.04043263 0.06748637 0.09166412 ... 0.10462585 0.         0.04567996]
 ...
 [0.07251541 0.06275192 0.06756014 ... 0.07906516 0.         0.05174178]
 [0.05823517 0.05963557 0.09027178 ... 0.10724749 0.         0.0523259 ]
 [0.12807868 0.10718347 0.12245025 ... 0.07215846 0.         0.03251233]]


In [4]:
print(Dataset.latent_train.shape)

(200, 300)


In [5]:
class Params:
    latent_feature_count = [10,10,3]
    epochs = 1
    input_batch_size = 200
    output_size = 1

In [6]:
class Network:
    
    def __init__(self):
        self.latent = tf.placeholder(tf.float32, shape=[Params.input_batch_size] + [np.prod(Params.latent_feature_count)], name='x')
        print('Input Latent', self.latent.get_shape())
        hidden_state = tf.zeros([Params.output_size, np.prod(Params.latent_feature_count)])
        current_state = tf.zeros([Params.output_size, np.prod(Params.latent_feature_count)])
        self.state = hidden_state, current_state
        
        
    def build_network(self):
        
        y_list = []
        
        with tf.name_scope('LSTM') as lstm_scope:
            x = tf.layers.dense(self.latent, units=np.prod(Params.latent_feature_count), activation=tf.nn.relu)
            
            lstm = tf.contrib.rnn.BasicLSTMCell(512) #np.prod(Params.latent_feature_count))
            x_list = tf.split(x, 10, axis=0)
            #print("Mini Batch ", x_list.get_shape())
            

            y_list, self.state = tf.nn.static_rnn(lstm, x_list, dtype=tf.float32)
            
            self.y = y_list[-1]
            self.y = tf.layers.dense(self.y, np.prod(Params.latent_feature_count))
                
            print('Output Latent', self.y.get_shape())
            
        with tf.name_scope('Weights'):
            n = Params.input_batch_size
            f = np.prod(Params.latent_feature_count)

            X = self.y

            P = tf.Variable(tf.truncated_normal([f]))
            Q_ = tf.Variable(tf.truncated_normal([f])) 
            
        with tf.name_scope('RBF'):
            Q = tf.diag(tf.square(Q_)) + tf.eye(f) 
            #Q = tf.matmul(Q_, tf.transpose(Q_)) # Making is positive semi-definite

            X = tf.layers.dense(X, f)
            X = tf.layers.dense(X, f)

            M = P - tf.matmul(X, Q)
            print('M', M.get_shape())

            R = tf.matmul(M, Q)
            print('R', R.get_shape())

            V = tf.matmul(R, tf.transpose(M))
            print('V', V.get_shape())

            U = tf.reduce_sum(V, axis = 1, keepdims = True)
            print('U', U.get_shape())

            Q_det = tf.matrix_determinant(Q)
            print('Q_det', Q_det.get_shape())

            Z = tf.pow(tf.multiply(tf.pow(2*math.pi,f), Q_det), 1/2)
            print('Z', Z.get_shape())
        
        with tf.name_scope('Likelihood'):
            Y_ = tf.divide(tf.exp(-U/2), Z)
            print("Y_", Y_.get_shape())
            Y_ = tf.layers.dense(Y_, 1)
            self.Y_ = tf.layers.dense(Y_, 1)
            
        with tf.name_scope('LossAndOptim'):
            global_step = tf.Variable(0, trainable=False)
            
            cost = - (-U - tf.log(Z))
            loss = tf.reduce_mean(cost)

            starter_learning_rate = 0.01
            learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                               1000, 0.5, staircase=True)
            self.train = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)


In [7]:
with tf.Session() as sess:
    network=Network()
    network.build_network()
    sess.run(tf.global_variables_initializer())
    
    for i in range(2): # crashes in 3rd epoch
        Dataset.likelihood, _, Dataset.latent = sess.run((network.Y_, network.train, network.y), 
                                                     feed_dict={network.latent:Dataset.latent_train})

Input Latent (200, 300)
Output Latent (20, 300)
M (20, 300)
R (20, 300)
V (20, 20)
U (20, 1)
Q_det ()
Z ()
Y_ (20, 1)


In [8]:
Dataset.likelihood

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]], dtype=float32)

In [9]:
Dataset.likelihood.shape

(20, 1)

In [10]:
Dataset.latent

array([[-0.65905994,  2.7989202 ,  1.6895077 , ..., -1.9592539 ,
        -3.9366913 , -1.9573234 ],
       [-0.66371995,  2.805173  ,  1.693853  , ..., -1.9630744 ,
        -3.9431162 , -1.9619129 ],
       [-0.6604975 ,  2.8030047 ,  1.6921544 , ..., -1.9615117 ,
        -3.940509  , -1.9605842 ],
       ...,
       [-0.6623093 ,  2.8013177 ,  1.6893344 , ..., -1.9615673 ,
        -3.9382448 , -1.9587536 ],
       [-0.6580329 ,  2.7942502 ,  1.6845864 , ..., -1.9558983 ,
        -3.9288182 , -1.952956  ],
       [-0.6653598 ,  2.8128011 ,  1.698145  , ..., -1.9678222 ,
        -3.951354  , -1.9664912 ]], dtype=float32)

In [11]:
Dataset.latent.shape

(20, 300)